In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date
from unidecode import unidecode

BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Raw data

In [32]:
df = pd.read_csv(f"{BASE}work_data/matriz_desercion_3.csv.gz")
print(df.columns)
df.head()

/tmp/ipykernel_25632/3856008204.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{BASE}work_data/matriz_desercion_3.csv.gz")


Index(['cat_periodo', 'codigo_unico', 'mrun', 'gen_alu', 'fec_nac_alu',
       'anio_ing_carr_ori', 'tipo_inst_1', 'region_sede', 'valor_arancel',
       'area_conocimiento', 'acreditada_carr', 'acreditada_inst', 'desertor_1',
       'region_colegio', 'tipo_ensenanza_colegio', 'prom_notas_media',
       'dependencia_colegio', 'fscu', 'gratuidad', 'beca'],
      dtype='object')


,cat_periodo,codigo_unico,mrun,gen_alu,fec_nac_alu,anio_ing_carr_ori,tipo_inst_1,region_sede,valor_arancel,area_conocimiento,acreditada_carr,acreditada_inst,desertor_1,region_colegio,tipo_ensenanza_colegio,prom_notas_media,dependencia_colegio,fscu,gratuidad,beca
0,2015,I498S6C132J2V1,37.0,1,197808,2015,Centros de Formación Técnica,Biobío,820000.0,Tecnología,NO ACREDITADA,ACREDITADA,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,2015,I111S14C309J2V1,118.0,1,199102,2015,Institutos Profesionales,Metropolitana,1390000.0,Administración y Comercio,ACREDITADA,ACREDITADA,0.0,13,310.0,4.975,3.0,0.0,0.0,0.0
2,2015,I106S3C64J4V1,253.0,1,198404,2015,Institutos Profesionales,Metropolitana,920000.0,Administración y Comercio,NO ACREDITADA,NO ACREDITADA,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,2015,I221S1C57J1V1,370.0,2,198910,2015,Centros de Formación Técnica,Metropolitana,1880000.0,Administración y Comercio,ACREDITADA,ACREDITADA,1.0,13,310.0,5.100,3.0,0.0,0.0,1.0
4,2015,I176S8C59J2V1,533.0,2,198309,2015,Institutos Profesionales,Coquimbo,1190000.0,Salud,ACREDITADA,NO ACREDITADA,0.0,4,510.0,5.200,2.0,0.0,0.0,0.0


## Selección periodo
- `cat_periodo` entre 2015 y 2019 inclusive. 

In [33]:
df = df[df["cat_periodo"] < 2020]

In [34]:
df_mrun = df[["cat_periodo", "mrun"]].drop_duplicates()
df_mrun.shape

(1797155, 2)

In [36]:
df_mrun.to_csv(f"{BASE}work_data/mrun_anio.csv", index=False)

# Nulos

In [4]:
df["fec_nac_alu"] = df["fec_nac_alu"].replace([190001], [np.nan])
df["anio_ing_carr_ori"] = df["anio_ing_carr_ori"].replace([9995], [np.nan])
df["acreditada_carr"] = df["acreditada_carr"].str.strip().replace(["", "SIN INFORMACION"], [np.nan, np.nan])
df["acreditada_inst"] = df["acreditada_inst"].str.strip().replace([""], [np.nan])
print(df.shape)
df = df.dropna()
print(df.shape)

(1802594, 20)
(1560678, 20)


# Pre procesamiento variables dependientes datos desagregados
## Diferencia año ingreso y último año de matrícula

In [5]:
g = df.groupby(["mrun", "codigo_unico"]).agg({
    "cat_periodo": "max",
    "anio_ing_carr_ori": "min"
}).reset_index()
g["dif_ing_ult_mat"] = g["cat_periodo"] - g["anio_ing_carr_ori"]
g.head()

,mrun,codigo_unico,cat_periodo,anio_ing_carr_ori,dif_ing_ult_mat
0,19.0,I536S0C39J1V1,2019,2019,0
1,118.0,I111S14C309J2V1,2015,2015,0
2,118.0,I221S1C113J2V1,2017,2015,2
3,196.0,I280S8C380J1V1,2019,2017,2
4,238.0,I143S15C614J1V1,2017,2016,1


In [6]:
if "dif_ing_ult_mat" not in df.columns:
    print(df.shape)
    df = pd.merge(
        df,
        g[["mrun", "codigo_unico", "dif_ing_ult_mat"]],
        how="left"
    )
    print(df.shape)

(1560678, 20)
(1560678, 21)


In [7]:
df["dif_ing_ult_mat"].value_counts("%") * 100

2    31.432557
1    28.824972
0    23.873599
3    12.011895
4     3.856978
Name: dif_ing_ult_mat, dtype: float64

## Edad

In [8]:
# Se pisa año de ingreso por el minimo
print(df.shape)
df = pd.merge(
    df.drop(columns=["anio_ing_carr_ori"]),
    g[["mrun", "codigo_unico", "anio_ing_carr_ori"]],
    how="left"
)
df.shape

(1560678, 21)


(1560678, 21)

In [9]:
df["fecha_n"] = pd.to_datetime([
    date(int(str(d)[:4]), int(str(d)[-2:]), 15)
    for d in df["fec_nac_alu"].astype(int)
])

df["Edad"] = [
    date(y, 1, 1).year - fn.year - (
        (date(y, 1, 1).month, date(y, 1, 1).day) < (fn.month, fn.day)
    )
    for fn, y in zip(df["fecha_n"], df["anio_ing_carr_ori"])
]
df.shape

(1560678, 23)

# Registros únicos

- Se crea 1 registro por combinación `mrun` - `codigo_unico`.

In [10]:
for c in ["cat_periodo", "anio_ing_carr_ori", "fecha_n", "fec_nac_alu"]:
    if c in df.columns:
        df = df.drop(columns=[c])
        
df.shape

(1560678, 19)

In [11]:
# Se mantiene el registro donde esté como desertor
t = df.drop_duplicates().sort_values(by=["mrun", "codigo_unico", "desertor_1"], ascending=[False, False, False]).drop_duplicates(subset=["mrun", "codigo_unico"], keep="first").copy()
t.shape

(885787, 19)

In [12]:
# No debe haber mrun-codigo_unico repetidos
assert t[["mrun", "codigo_unico"]].value_counts().reset_index()[0].max() == 1

In [13]:
# Se revisa dif
t["dif_ing_ult_mat"].value_counts("%") * 100

0    42.063160
1    27.811765
2    21.017920
3     6.998522
4     2.108633
Name: dif_ing_ult_mat, dtype: float64

In [14]:
print("Desertores")
print(t[t["desertor_1"]==1]["dif_ing_ult_mat"].value_counts("%") * 100)
print("\nNo desetores")
print(t[t["desertor_1"]==0]["dif_ing_ult_mat"].value_counts("%") * 100)

Desertores
0    58.734491
1    24.369195
2    11.744173
3     4.046734
4     1.105406
Name: dif_ing_ult_mat, dtype: float64

No desetores
0    36.789593
1    28.900738
2    23.951442
3     7.932248
4     2.425980
Name: dif_ing_ult_mat, dtype: float64


In [15]:
t["desertor_1"].value_counts()

0.0    672924
1.0    212863
Name: desertor_1, dtype: int64

# Pre procesamiento resto de variables
## Tipo enseñanza

In [16]:
omit_dep = [
    'LOS ANDES', 'SANTIAGO ORIENTE', 'VALPARAISO', 'SANTIAGO PONIENTE',
    'COLCHAGUA', 'VALPARAÍSO', 'CAUTÍN NORTE', 'ANTOFAGASTA - TOCOPILLA'
]

t = t[~t["dependencia_colegio"].isin(omit_dep)]
t.shape

(885586, 19)

In [17]:
t["dependencia_colegio"] = t["dependencia_colegio"].astype(int)
t["dependencia_colegio"].unique()

array([3, 2, 1, 6, 5, 4])

In [18]:
t["dependencia_colegio"] = t["dependencia_colegio"].map({
    1: 'Municipal',
    2: 'Municipal',
    3: 'Particular Subvencionado',
    4: 'Particular Pagado',
    5: 'Otro',
    6: 'Otro'
})
t["dependencia_colegio"].value_counts()

Particular Subvencionado    444148
Municipal                   361087
Otro                         50947
Particular Pagado            29404
Name: dependencia_colegio, dtype: int64

## Dependencia Colegio

In [19]:
t["tipo_ensenanza_colegio"] = t["tipo_ensenanza_colegio"].map({
    0: "no_aplica",
    10: "Parvularia",
    110: "Basica",
    160: "Basica",
    161: "Basica",
    163: "Carceles",
    165: "Basica",
    167: "Basica",
    211: "Especial",
    212: "Especial",
    213: "Especial",
    214: "Especial",
    215: "Especial",
    216: "Especial",
    217: "Especial",
    299: "PIE",
    310: "HC",
    360: "HC",
    361: "HC",
    362: "Carceles",
    363: "HC",
    410: "TP Comercial",
    460: "TP Comercial",
    461: "TP Comercial",
    463: "TP Comercial",
    510: "TP Industrial",
    560: "TP Industrial",
    561: "TP Industrial",
    563: "TP Industrial",
    610: "TP Tecnica",
    660: "TP Tecnica",
    661: "TP Tecnica",
    663: "TP Tecnica",
    710: "Otro",
    760: "Otro",
    761: "Otro",
    763: "Otro",
    810: "Otro",
    860: "Otro",
    863: "Otro",
    910: "Otro",
    963: "Otro"                     
})

t["tipo_ensenanza_colegio"].value_counts()

HC               500449
TP Comercial     143680
TP Industrial    140212
TP Tecnica        82524
Otro              18721
Name: tipo_ensenanza_colegio, dtype: int64

## Región colegio

In [20]:
t["region_colegio"] = t["region_colegio"].astype(int)
t["region_colegio"].unique()

array([13,  7,  2,  6,  9,  5, 10, 16,  4,  1, 14, 15, 12,  8, 11,  3])

In [21]:
t["region_colegio"] = t["region_colegio"].map({
    1: "Tarapacá",
    2: "Antofagasta",
    3: "Atacama",
    4: "Coquimbo",
    5: "Valparaíso",
    6: "Lib. Gral B. O'Higgins",
    7: "Maule",
    8: "Biobío",
    9: "La Araucanía",
    10: "Los Lagos",
    11: "Aysén",
    12: "Magallanes",
    13: "Metropolitana",
    14: "Los Ríos",
    15: "Arica y Parinacota",
    16: "Ñuble"
})
t["region_colegio"].value_counts()

Metropolitana             354429
Valparaíso                 91111
Biobío                     80667
Maule                      51746
Lib. Gral B. O'Higgins     49053
La Araucanía               46539
Coquimbo                   40650
Los Lagos                  39956
Antofagasta                32651
Ñuble                      21082
Los Ríos                   19441
Atacama                    15566
Tarapacá                   13626
Arica y Parinacota         13157
Magallanes                  9534
Aysén                       6378
Name: region_colegio, dtype: int64

In [22]:
t["misma_region"] = np.where(t["region_colegio"]==t["region_sede"], 1, 0)
t["misma_region"].value_counts("%") * 100

1    85.803637
0    14.196363
Name: misma_region, dtype: float64

## Genero, desertor, beneficios

In [23]:
t["gen_alu"] = t["gen_alu"].replace([1, 2], ["hombre", "mujer"])
t["desertor_1"] = t["desertor_1"].astype(int)
t["fscu"] = t["fscu"].astype(int)
t["gratuidad"] = t["gratuidad"].astype(int)
t["beca"] = t["beca"].astype(int)
t["gen_alu"].value_counts()

hombre    443210
mujer     442376
Name: gen_alu, dtype: int64

# No dummy

In [28]:
t.head()

,codigo_unico,mrun,gen_alu,tipo_inst_1,region_sede,valor_arancel,area_conocimiento,acreditada_carr,acreditada_inst,desertor_1,region_colegio,tipo_ensenanza_colegio,prom_notas_media,dependencia_colegio,fscu,gratuidad,beca,dif_ing_ult_mat,Edad,misma_region
1560677,I426S1C7J2V2,25866904.0,mujer,Centros de Formación Técnica,Metropolitana,1390000.0,Administración y Comercio,NO ACREDITADA,ACREDITADA,1,Metropolitana,HC,6.4,Particular Subvencionado,0,0,0,0,35,1
1560676,I143S26C376J2V1,25768831.0,hombre,Institutos Profesionales,Metropolitana,1850000.0,Tecnología,ACREDITADA,ACREDITADA,1,Metropolitana,HC,5.5,Particular Subvencionado,0,0,1,0,31,1
1560675,I273S1C5J2V1,25768815.0,hombre,Centros de Formación Técnica,Maule,803000.0,Tecnología,NO ACREDITADA,NO ACREDITADA,0,Maule,HC,4.7,Particular Subvencionado,0,0,0,0,29,1
1560674,I143S29C26J2V1,25768749.0,hombre,Institutos Profesionales,Metropolitana,1690000.0,Tecnología,NO ACREDITADA,ACREDITADA,0,Metropolitana,HC,6.0,Municipal,0,0,0,0,29,1
1560673,I143S15C30J1V1,25768671.0,mujer,Institutos Profesionales,Metropolitana,1480000.0,Derecho,NO ACREDITADA,ACREDITADA,0,Metropolitana,HC,6.4,Particular Subvencionado,0,0,0,0,16,1


In [30]:
t.to_csv(f"{BASE}work_data/matriz_desercion_3_no_dummy.csv.gz", index=False, compression="gzip")

# Dummys

In [24]:
t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 885586 entries, 1560677 to 1144184
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   codigo_unico            885586 non-null  object 
 1   mrun                    885586 non-null  float64
 2   gen_alu                 885586 non-null  object 
 3   tipo_inst_1             885586 non-null  object 
 4   region_sede             885586 non-null  object 
 5   valor_arancel           885586 non-null  float64
 6   area_conocimiento       885586 non-null  object 
 7   acreditada_carr         885586 non-null  object 
 8   acreditada_inst         885586 non-null  object 
 9   desertor_1              885586 non-null  int64  
 10  region_colegio          885586 non-null  object 
 11  tipo_ensenanza_colegio  885586 non-null  object 
 12  prom_notas_media        885586 non-null  float64
 13  dependencia_colegio     885586 non-null  object 
 14  fscu         

In [25]:
dummy = pd.get_dummies(t.drop(columns=["codigo_unico", "mrun", "region_colegio"]), drop_first=True)
dummy.head()

,valor_arancel,desertor_1,prom_notas_media,fscu,gratuidad,beca,dif_ing_ult_mat,Edad,misma_region,gen_alu_mujer,...,area_conocimiento_Tecnología,acreditada_carr_NO ACREDITADA,acreditada_inst_NO ACREDITADA,tipo_ensenanza_colegio_Otro,tipo_ensenanza_colegio_TP Comercial,tipo_ensenanza_colegio_TP Industrial,tipo_ensenanza_colegio_TP Tecnica,dependencia_colegio_Otro,dependencia_colegio_Particular Pagado,dependencia_colegio_Particular Subvencionado
1560677,1390000.0,1,6.4,0,0,0,0,35,1,1,...,0,1,0,0,0,0,0,0,0,1
1560676,1850000.0,1,5.5,0,0,1,0,31,1,0,...,1,0,0,0,0,0,0,0,0,1
1560675,803000.0,0,4.7,0,0,0,0,29,1,0,...,1,1,1,0,0,0,0,0,0,1
1560674,1690000.0,0,6.0,0,0,0,0,29,1,0,...,1,1,0,0,0,0,0,0,0,0
1560673,1480000.0,0,6.4,0,0,0,0,16,1,1,...,0,1,0,0,0,0,0,0,0,1


In [26]:
dummy.columns = [
    unidecode(c.replace(" ", "_").replace(".", "").replace("'", "").lower())
    for c in dummy.columns
]

dummy.columns

Index(['valor_arancel', 'desertor_1', 'prom_notas_media', 'fscu', 'gratuidad',
       'beca', 'dif_ing_ult_mat', 'edad', 'misma_region', 'gen_alu_mujer',
       'tipo_inst_1_institutos_profesionales',
       'region_sede_arica_y_parinacota', 'region_sede_atacama',
       'region_sede_aysen', 'region_sede_biobio', 'region_sede_coquimbo',
       'region_sede_la_araucania', 'region_sede_lib_gral_b_ohiggins',
       'region_sede_los_lagos', 'region_sede_los_rios',
       'region_sede_magallanes', 'region_sede_maule',
       'region_sede_metropolitana', 'region_sede_tarapaca',
       'region_sede_valparaiso', 'region_sede_nuble',
       'area_conocimiento_agropecuaria',
       'area_conocimiento_arte_y_arquitectura',
       'area_conocimiento_ciencias_basicas',
       'area_conocimiento_ciencias_sociales', 'area_conocimiento_derecho',
       'area_conocimiento_educacion', 'area_conocimiento_humanidades',
       'area_conocimiento_salud', 'area_conocimiento_tecnologia',
       'acreditada_ca

- En género elimina hombre.
- En tipo institución elimina CFT.
- En `region_sede` elimina Antofagasta.
- En `area_conocimiento` elimina Administración y Comercio.
- En ambas acreditaciones elimina Acreditada.
- En `tipo_ensenanza_colegio` elimina HC.
- En `dependencia_colegio` elimina Municipal.

In [27]:
dummy.to_csv(f"{BASE}work_data/matriz_desercion_3_dummy.csv.gz", index=False, compression="gzip")